In [3]:
import os
import torch
import pandas as pd
import numpy as np
import tldextract
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification

In [4]:
# Create directories for saving models
SAVE_DIR = "saved_model_fakenewsnet_bert_metadata"
os.makedirs(SAVE_DIR, exist_ok=True)

# =============================================================================
# Step 1: Load and Prepare Data (using all meta data)
# =============================================================================
# Replace these paths with your actual file locations.
fake_news = pd.read_csv('data/fake_with_metadata.csv')
real_news = pd.read_csv('data/real_with_metadata.csv')

# Drop unnecessary columns if they exist
fake_news.drop(columns=['id'], errors='ignore', inplace=True)
real_news.drop(columns=['id'], errors='ignore', inplace=True)

# Assign labels: Here we assume 1 for Fake and 0 for Real
fake_news['labels'] = 1
real_news['labels'] = 0

# -----------------------------------------------------
# Combine data. Here we assume that your CSVs contain
# meta columns like 'title', 'meta_title', 'meta_description'
# and 'news_url'. Adjust the list of columns if needed.
# -----------------------------------------------------
meta_cols = ['title', 'meta_title', 'meta_description', 'news_url']
# If your CSV files have these columns, select them.
fake_selected = fake_news[meta_cols + ['labels']]
real_selected = real_news[meta_cols + ['labels']]

data = pd.concat([fake_selected, real_selected], ignore_index=True)

# Drop any missing values in the main text field (title)
data.dropna(subset=['title'], inplace=True)
# Fill missing values for meta columns (if any)
for col in ['meta_title', 'meta_description', 'news_url']:
    data[col] = data[col].fillna("")

# =============================================================================
# Step 2: Combine Meta Data Text into a Single Field
# =============================================================================
def combine_metadata(row):
    # Combine title, meta_title, and meta_description
    texts = [str(row.get(col, "")) for col in ['title', 'meta_title', 'meta_description']]
    return " ".join(texts).strip()

data['combined_text'] = data.apply(combine_metadata, axis=1)
# Now you have one text field to represent all your text meta data.

# =============================================================================
# Step 3: Extract URL Features
# =============================================================================
def extract_url_features(url):
    if pd.isna(url) or not isinstance(url, str):
        return {'url_length': 0, 'has_query': 0, 'has_hyphens': 0, 'has_numbers': 0}
    
    return {
        'url_length': len(url),
        'has_query': 1 if '?' in url else 0,
        'has_hyphens': 1 if '-' in url else 0,
        'has_numbers': 1 if any(c.isdigit() for c in url) else 0,
    }

url_features = data['news_url'].apply(extract_url_features)
url_features_df = pd.DataFrame(url_features.tolist())

In [6]:
# =============================================================================
# Step 4: Vectorize Combined Meta Data Text using TF-IDF
# =============================================================================
# Note: Instead of only the title, we now vectorize the combined text.
vectorizer = TfidfVectorizer(max_features=5000)
meta_text_features_vectorized = vectorizer.fit_transform(data['combined_text']).toarray()

# =============================================================================
# Step 5: Normalize URL Features
# =============================================================================
scaler = StandardScaler()
url_features_scaled = pd.DataFrame(scaler.fit_transform(url_features_df), 
                                   columns=url_features_df.columns)

# Save the scaler and vectorizer
torch.save(scaler, os.path.join(SAVE_DIR, "scaler.pth"))
torch.save(vectorizer, os.path.join(SAVE_DIR, "vectorizer.pth"))
print("Saved scaler and vectorizer")

# =============================================================================
# Step 6: Align Data Dimensions
# =============================================================================
# Since all our feature extraction comes from the same DataFrame, ensure alignment
min_length = min(len(meta_text_features_vectorized), len(url_features_scaled))
meta_text_features_vectorized = meta_text_features_vectorized[:min_length]
url_features_scaled = url_features_scaled.iloc[:min_length]
data = data.iloc[:min_length]

# =============================================================================
# Step 7: Apply SMOTE to Handle Class Imbalance
# =============================================================================
combined_features = np.hstack([meta_text_features_vectorized, url_features_scaled])
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(combined_features, data['labels'])

# =============================================================================
# Step 8: Split Data into Training & Testing Sets
# =============================================================================
num_text_features = meta_text_features_vectorized.shape[1]
train_texts, test_texts, train_labels, test_labels, train_urls, test_urls = train_test_split(
    X_resampled[:, :num_text_features], 
    y_resampled, 
    X_resampled[:, num_text_features:], 
    test_size=0.2, 
    random_state=42
)

# Convert labels and URL features to tensors using the .values attribute or np.array()
train_labels = torch.tensor(train_labels.values, dtype=torch.long)
test_labels = torch.tensor(test_labels.values, dtype=torch.long)
train_urls = torch.tensor(train_urls, dtype=torch.float)
test_urls = torch.tensor(test_urls, dtype=torch.float)

# =============================================================================
# Step 9: Tokenize Text for BERT
# =============================================================================
# Since we used SMOTE on TF-IDF features, we now need to convert those back to text.
# We use the vectorizer's inverse_transform to extract tokens, then join them into strings.
train_text_strings = [
    " ".join(vectorizer.inverse_transform(text_row.reshape(1, -1))[0])
    for text_row in train_texts
]
test_text_strings = [
    " ".join(vectorizer.inverse_transform(text_row.reshape(1, -1))[0])
    for text_row in test_texts
]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(
    train_text_strings,
    truncation=True, padding=True, max_length=512, return_tensors="pt"
)
test_encodings = tokenizer(
    test_text_strings,
    truncation=True, padding=True, max_length=512, return_tensors="pt"
)

# Save the tokenizer
tokenizer.save_pretrained(SAVE_DIR)
print("Saved tokenizer")


Saved scaler and vectorizer


C:\Users\jun_k\anaconda3\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\jun_k\anaconda3\lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Saved tokenizer


In [7]:
# =============================================================================
# Step 10: Define Custom Dataset Class for Combined Features
# =============================================================================
class CombinedDataset(Dataset):
    def __init__(self, encodings, labels, url_features):
        self.encodings = encodings
        self.labels = labels
        self.url_features = url_features

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        item['url_features'] = self.url_features[idx]
        return item

# Create datasets and dataloaders
train_dataset = CombinedDataset(train_encodings, train_labels, train_urls)
test_dataset = CombinedDataset(test_encodings, test_labels, test_urls)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_dataloader = DataLoader(test_dataset, batch_size=16)

# =============================================================================
# Step 11: Define the BertWithFeatures Model (including URL features)
# =============================================================================
class BertWithFeatures(nn.Module):
    def __init__(self, num_labels=2, url_feature_dim=4):  
        super(BertWithFeatures, self).__init__()
        # Load BERT for classification
        self.bert = BertForSequenceClassification.from_pretrained(
            "bert-base-uncased", num_labels=num_labels
        )
        # A fully-connected layer for URL features
        self.url_fc = nn.Linear(url_feature_dim, 32)  
        self.dropout = nn.Dropout(0.3)
        # Final combined classifier (768 from BERT pooler + 32 URL features)
        self.fc = nn.Linear(768 + 32, num_labels)

    def forward(self, input_ids, attention_mask, url_features):
        outputs = self.bert.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  
        url_emb = torch.relu(self.url_fc(url_features))
        combined = torch.cat((pooled_output, url_emb), dim=1)
        combined = self.dropout(combined)
        logits = self.fc(combined)
        return logits

In [ ]:
# =============================================================================
# Step 12: Train the Model
# =============================================================================
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = BertWithFeatures().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
loss_fct = nn.CrossEntropyLoss()
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    loop = tqdm(train_dataloader, leave=True)
    for batch in loop:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(batch['input_ids'], batch['attention_mask'], batch['url_features'])
        loss = loss_fct(logits, batch['labels'])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loop.set_description(f"Epoch {epoch+1}/{num_epochs}")
        loop.set_postfix(loss=loss.item())

# Save the trained model
model_save_path = os.path.join(SAVE_DIR, "bert_fakenewsnet_model.pth")
torch.save(model.state_dict(), model_save_path)
print("Saved model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 2/5:  21%|██████████▉                                        | 373/1745 [3:12:29<12:10:32, 31.95s/it, loss=0.185]

In [ ]:
# =============================================================================
# Step 13: Evaluate the Model
# =============================================================================
model.eval()
predictions, true_labels = [], []
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        logits = model(batch['input_ids'], batch['attention_mask'], batch['url_features'])
    preds = torch.argmax(logits, dim=1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(batch["labels"].cpu().numpy())

# Display evaluation metrics
print("Accuracy:", accuracy_score(true_labels, predictions))
print(classification_report(true_labels, predictions, target_names=['Real', 'Fake']))